In [1]:
from pandas.tseries.offsets import DateOffset
import ee
import yaml
import time
import os
import json
import pandas as pd
from utils import exportImageToGDrive,exportImageToGCS
from download_sen12 import *


ee.Initialize()

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][0]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)

project-canopy-temp-2


C:\Users\David\canopy\canopy-gis\google-earth-explore\GEE_DataDownloader\download_sen12.py:401: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [3]:
json_file_path = 'tiles_polygons_full_v2.json'

with open(json_file_path, 'r') as j:
     tile_hash = json.loads(j.read())

In [6]:
def import_aois(csv_loc, Full_Congo_Pull=False, start_date=None,
                end_date=None, days_duration=90, poly_start=0, poly_limit=None):
    features = []
    polygons = []
    day_offset = days_duration / 2
    start_end_list = []
    
    if Full_Congo_Pull:
        df_congo_tile_list = pd.read_csv(csv_loc)
        for index,tile,polygon in df_congo_tile_list.itertuples():
            poly_obj = ee.Geometry.Polygon(json.loads(polygon))
            feature = ee.Feature(poly_obj,{"name":tile})
            features.append(feature)
    else:
        feature_id = 0 
        
        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined","tile date"]]
        df_labels["tile date"] = pd.to_datetime(df_labels["tile date"])
        start = (df_labels["tile date"] + DateOffset(days=-day_offset))
        end = (df_labels["tile date"] + DateOffset(days=day_offset))
        for i in range(len(start)):
            start_date = str(start[i])[:10]
            end_date = str(end[i])[:10]
            original_date = df_labels.loc[i, 'tile date']
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': day_offset
            }
            start_end_list.append(date_dict)

        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])
            
        if poly_limit:
            polygons = polygons[poly_start:poly_start+poly_limit]  
        for poly in polygons[poly_start:]:
            # create an roi. first item in Misha's label list
            feature_id += 1 
            # create geometry object, create feature object, append to features list for feature collection creation 
            polys = ee.Geometry.Polygon(poly)
            feature = ee.Feature(polys,{"name":feature_id})
            features.append(feature)
            
    return ee.FeatureCollection(features),start_end_list,df_labels


# ### CHANGE BELOW PATH ###
zhenya_label_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"
david__label_path = 'D:/canopy_data/csvs/polygons_101320.csv'
#zhenya_tilelist_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tiles_polygons_full.csv"

In [7]:
fc,date_range_list,df_labels = import_aois(david__label_path)

In [8]:
date_range_list

[{'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0},
 {'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0},
 {'start_date': '2019-11-30',
  'end_date': '2020-02-28',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0},
 {'start_date': '2019-11-25',
  'end_date': '2020-02-23',
  'original_date': Timestamp('2020-01-09 00:00:00'),
  'day_offset': 45.0},
 {'start_date': '2019-11-30',
  'end_date': '2020-02-28',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0},
 {'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0},
 {'start_date': '2019-11-15',
  'end_date': '2020-02-13',
  'original_date': Timestamp('2019-12-30 00:00:00'),
  'day_offset': 45.0},
 {'start_date': '2020-11-15',
  'end_date': '2021-02-13',
  'o

In [4]:
original_date = pd.to_datetime('2020-01-01')
day_offset = 45

start = original_date + DateOffset(days=-day_offset)
end = original_date + DateOffset(days=day_offset)
start_date = str(start)[:10]
end_date = str(end)[:10]

tile_date_range = {
    'start_date': start_date,
    'end_date': end_date,
    'original_date': original_date,
    'day_offset': day_offset,
    'area': 100 * 100
}

tile_date_range

{'start_date': '2019-11-17',
 'end_date': '2020-02-15',
 'original_date': Timestamp('2020-01-01 00:00:00'),
 'day_offset': 45,
 'area': 10000}

In [5]:
date_range_list = [tile_date_range] * len(tile_hash)

len(date_range_list)

308

In [6]:
date_range_list[0:4]

[{'start_date': '2019-11-17',
  'end_date': '2020-02-15',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 45,
  'area': 10000},
 {'start_date': '2019-11-17',
  'end_date': '2020-02-15',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 45,
  'area': 10000},
 {'start_date': '2019-11-17',
  'end_date': '2020-02-15',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 45,
  'area': 10000},
 {'start_date': '2019-11-17',
  'end_date': '2020-02-15',
  'original_date': Timestamp('2020-01-01 00:00:00'),
  'day_offset': 45,
  'area': 10000}]

In [6]:
exports = process_datasource_custom_daterange(source, sensor, export_folder,
                                              tile_hash, date_range_list, limit=10)

308 tiles have been loaded
processing  1_v10_secondary_sort_no_dynamic_date_range_Dec__2_14:11:16_2020
Tile is 32MMD
not clipping in makeImageCollection
secondary sort -- getting infos


EEException: Computation timed out.

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
